In [1]:
%matplotlib notebook
from utils.dataloader import PascalVOCDataset
from utils.dataloader import myimshow
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision as tv
import numpy as np
from matplotlib import pyplot as plt
from utils.PositionSensitiveScoreMap import PositionSensitiveScoreMap

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
dataset_root_dir = "/datasets/ee285f-public/PascalVOC2012/"
# dataset_root_dir = "../Datasets/VOCtrainval_11-May-2012/VOCdevkit/VOC2012"

#Creating training and validation sets. Go to utils/dataloader.py for info on important PascalVOCDataset arguments & attributes
train_set = PascalVOCDataset(dataset_root_dir, mode= 'train')
val_set = PascalVOCDataset(dataset_root_dir, mode= 'val')

In [8]:
from __future__ import  absolute_import
# though cupy is not used but without this line, it raise errors...
import cupy as cp
import os

import matplotlib
from tqdm import tqdm

import torch as t
from utils.config import opt
from model.rfcn_resnet101 import RFCNResnet101
from torch.utils import data as data_
from trainer import FasterRCNNTrainer
from utils import array_tool as at
from utils.vis_tool import visdom_bbox
from utils.eval_tool import eval_detection_voc

# fix for ulimit
# https://github.com/pytorch/pytorch/issues/973#issuecomment-346405667
import resource

rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (20480, rlimit[1]))

matplotlib.use('agg')


def eval(dataloader, faster_rcnn, test_num=10000):
    gt_difficults = False
    pred_bboxes, pred_labels, pred_scores = list(), list(), list()
    gt_bboxes, gt_labels, gt_difficults = list(), list(), list()
    for ii, (imgs, sizes, gt_bboxes_, gt_labels_) in tqdm(enumerate(dataloader)):
        pred_bboxes_, pred_labels_, pred_scores_ = faster_rcnn.predict(imgs)
        gt_bboxes += list(gt_bboxes_.numpy())
        gt_labels += list(gt_labels_.numpy())
        pred_bboxes += pred_bboxes_
        pred_labels += pred_labels_
        pred_scores += pred_scores_
        if ii == test_num: break

    result = eval_detection_voc(
        pred_bboxes, pred_labels, pred_scores,
        gt_bboxes, gt_labels, gt_difficults,
        use_07_metric=True)
    return result


def train(train_set, val_set, load_path = False, epochs = 1, lr=1e-3, record_every = 300, lr_decay = 1e-3,test_num=500):

    train_dataloader = td.DataLoader(train_set, batch_size = 1, pin_memory = False, shuffle = True)
    test_dataloader = td.DataLoader(val_set, batch_size = 1, pin_memory = True)
    faster_rcnn = RFCNResnet101().cuda()
    print('model construct completed')
    trainer = FasterRCNNTrainer(faster_rcnn).cuda()
    saved_loss = []
    iterations = []
    if load_path:
        trainer.load(load_path)
        print('load pretrained model from %s' % load_path)
        state_dict = t.load(load_path)
        saved_loss = state_dict['losses']
        iterations = state_dict['iterations']
        
    best_map = 0
    lr_ = lr
    for epoch in range(epochs):
        trainer.reset_meters()
        for ii, (img, bbox_, label_, scale) in tqdm(enumerate(train_dataloader)):
            scale = at.scalar(scale)
            img, bbox, label = img.cuda().float(), bbox_.cuda(), label_.cuda()
            losses = trainer.train_step(img, bbox, label, scale)
            loss_info = 'Iter {}; Losses: RPN loc {}, RPN cls: {}, ROI loc {}, ROI cls {}, Total:{}'.format(
                                                str(ii),
                                                "%.3f" % losses[0].cpu().data.numpy(),
                                                "%.3f" % losses[1].cpu().data.numpy(),
                                                "%.3f" % losses[2].cpu().data.numpy(),
                                                "%.3f" % losses[3].cpu().data.numpy(),                                
                                                "%.3f" % losses[4].cpu().data.numpy())
            print(loss_info)
            if (ii + 1) % record_every == 0:
                
                iterations.append(ii + 1) 
                saved_loss.append([losses[0].cpu().item(),losses[1].cpu().item(),
                              losses[2].cpu().item(),losses[3].cpu().item(),
                              losses[4].cpu().item()])
                kwargs = {"losses": saved_loss, "iterations": iterations}
                trainer.save(saved_loss = saved_loss, iterations = iterations)
                print("new model saved")

In [9]:
'''
CHANGE load_path TO THE PATH TO YOUR CHECKPOINT IF YOU ARE RESUMING TRAINING. "checkpoints/rfcnBLAHBLAHBLAH"
'''
train(train_set, val_set, load_path = False, epochs = 1, lr=1e-3, record_every = 300)
#train(train_set, val_set, load_path = "checkpoints/...", epochs = 3, lr=1e-3, record_every = 300)

/datasets/home/home-02/50/150/prvo/Multi-Object-Detection/model/rfcn_resnet101.py:83: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(self.cls_layer.weight.data, 0.0, 0.01)
/datasets/home/home-02/50/150/prvo/Multi-Object-Detection/model/rfcn_resnet101.py:87: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(self.reg_layer.weight.data, 0.0, 0.01)

0it [00:00, ?it/s]

model construct completed


/datasets/home/home-02/50/150/prvo/Multi-Object-Detection/utils/PositionSensitiveScoreMap_V2.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  scores = self.softmax(F.adaptive_avg_pool2d(pooling_track.float(),(1,1))[:,:])

1it [00:00,  2.29it/s]

Iter 0; Losses: RPN loc 2.196, RPN cls: 0.340, ROI loc 8.534, ROI cls 2.994, Total:14.063


/datasets/home/home-02/50/150/prvo/Multi-Object-Detection/model/utils/bbox_tools.py:68: RuntimeWarning: overflow encountered in multiply
  h = xp.exp(dh) * src_height[:, xp.newaxis]

2it [00:01,  1.84it/s]

...


21it [01:04,  6.03s/it]

Iter 20; Losses: RPN loc 0.496, RPN cls: 0.821, ROI loc 17.071, ROI cls 2.135, Total:20.523



22it [01:11,  6.12s/it]

Iter 21; Losses: RPN loc 1.234, RPN cls: 0.201, ROI loc 23.498, ROI cls 2.146, Total:27.080


KeyboardInterrupt: 

In [21]:
'''This will load the state dict which will allow you to obtain losses'''
#state_dict = t.load("checkpoints/...")
#saved_losses = state_dict["other_info"]["saved_loss"]
#iterations = state_dict["other_info"]["iterations"]

#There are other arguments than other_info.

5